In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_plant1 = pd.read_csv('../input/solar-power-generation-data/Plant_1_Generation_Data.csv')
df_sensor1 = pd.read_csv('../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')

In [ ]:
df_sensor1 = df_sensor1.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)


In [ ]:
df = df_plant1['DATE_TIME'].str.split(" ", n = 1, expand = True) 

# # making separate first name column from new data frame 
df_plant1["date"]= df[0] 
  
# # making separate last name column from new data frame 
df_plant1["time"]= df[1]

In [ ]:
df_plant1

In [ ]:
df_plant1['DATE_TIME'] = pd.to_datetime(df_plant1['DATE_TIME'],format='%d-%m-%Y %H:%M')
df_sensor1['DATE_TIME'] = pd.to_datetime(df_sensor1['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
print(max(df_plant1[(df_plant1['DC_POWER']!=0) & (df_plant1['AC_POWER']!=0)]['DATE_TIME']))
print(min(df_plant1[(df_plant1['DC_POWER']!=0) & (df_plant1['AC_POWER']!=0)]['DATE_TIME']))

In [ ]:
df_plant = df_plant1[(6 <= df_plant1['DATE_TIME'].dt.hour) & (df_plant1['DATE_TIME'].dt.hour < 19)].set_index('DATE_TIME')
df_sensor1 = df_sensor1[(6 <= df_sensor1['DATE_TIME'].dt.hour) & (df_sensor1['DATE_TIME'].dt.hour < 19)].set_index('DATE_TIME')

data1 = df_plant1.join(df_sensor1, on='DATE_TIME',how = 'inner')


In [ ]:
data1

In [ ]:
df_period = data1[['DATE_TIME','PLANT_ID']]
df_period = df_period.drop_duplicates('DATE_TIME')

s = []
for i in range(0, len(df_period)):
    s.append(i+1)

df_period['period'] = s
df_period = df_period[['DATE_TIME','period']] 

data1 = data1.merge(df_period, on ='DATE_TIME', how='left')
data1

In [ ]:
data1 = data1.groupby(['period','DATE_TIME','date','time']).mean().round(0).reset_index()
data1 = data1.drop(['PLANT_ID'], axis=1)
data1

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2,ax3, ax4,ax5) = plt.subplots(5)
fig.suptitle('Horizontally stacked subplots')
ax1.plot(data1["DATE_TIME"], data1["TOTAL_YIELD"])
ax2.plot(data1["DATE_TIME"], data1["DAILY_YIELD"])
ax3.plot(data1["DATE_TIME"], data1["AMBIENT_TEMPERATURE"])
ax4.plot(data1["DATE_TIME"], data1["MODULE_TEMPERATURE"])
ax5.plot(data1["DATE_TIME"], data1["IRRADIATION"])

# Extract feature

In [ ]:
data1.columns

In [ ]:
import copy 
df1 = copy.copy(data1)
df2 = copy.copy(data1)

# df = data1


# df['TOTAL_YIELD_1'] = df.groupby(['time']).shift(4)['TOTAL_YIELD'] # 1 วันก่อนหน้า
# df['TOTAL_YIELD_2'] = df.groupby(['DATE_TIME']).shift(5)['TOTAL_YIELD'] # 2 วันก่อนหน้า
# df['TOTAL_YIELD_3'] = df.groupby(['DATE_TIME']).shift(6)['TOTAL_YIELD'] # 3 วันก่อนหน้า

# df['TOTAL_YIELD_4'] = df.groupby(['DATE_TIME']).shift(7)['TOTAL_YIELD']
# df['TOTAL_YIELD_5'] = df.groupby(['DATE_TIME']).shift(8)['TOTAL_YIELD']
# df['TOTAL_YIELD_6'] = df.groupby(['DATE_TIME']).shift(9)['TOTAL_YIELD']
# df['TOTAL_YIELD_7'] = df.groupby(['DATE_TIME']).shift(10)['TOTAL_YIELD']


for i in range(0,3):
    
    df1['TOTAL_YIELD_'+str(i+1)] = df1.groupby(['time']).shift(i+4)['TOTAL_YIELD'] # 1 วันก่อนหน้า
    df1['DC_POWER_'+str(i+1)] = df1.groupby(['time']).shift(i+4)['DC_POWER'] # 1 วันก่อนหน้า
    df1['AC_POWER_'+str(i+1)] = df1.groupby(['time']).shift(i+4)['AC_POWER'] # 1 วันก่อนหน้า
    df1['AMBIENT_TEMPERATURE_'+str(i+1)] = df1.groupby(['time']).shift(i+4)['AMBIENT_TEMPERATURE'] # 1 วันก่อนหน้า
    df1['MODULE_TEMPERATURE_'+str(i+1)] = df1.groupby(['time']).shift(i+4)['MODULE_TEMPERATURE'] # 1 วันก่อนหน้า
    df1['IRRADIATION_'+str(i+1)] = df1.groupby(['time']).shift(i+4)['IRRADIATION'] # 1 วันก่อนหน้า

df1['mean_TOTAL_YIELD'] = (df1['TOTAL_YIELD_1'] + df1['TOTAL_YIELD_2'] + df1['TOTAL_YIELD_3'])/3
df1['mean_DC_POWER'] = (df1['DC_POWER_1'] + df1['DC_POWER_2'] + df1['DC_POWER_3'])/3
df1['mean_AC_POWER'] = (df1['AC_POWER_1'] + df1['AC_POWER_2'] + df1['AC_POWER_3'])/3
df1['mean_AMBIENT_TEMPERATURE'] = (df1['AMBIENT_TEMPERATURE_1'] + df1['AMBIENT_TEMPERATURE_2'] + df1['AMBIENT_TEMPERATURE_3'])/3
df1['mean_MODULE_TEMPERATURE'] = (df1['MODULE_TEMPERATURE_1'] + df1['MODULE_TEMPERATURE_2'] + df1['MODULE_TEMPERATURE_3'])/3
df1['mean_IRRADIATION'] = (df1['IRRADIATION_1'] + df1['IRRADIATION_2'] + df1['IRRADIATION_3'])/3


for j in range(0,7):
    df2['TOTAL_YIELD_'+str(j+1)] = df2.groupby(['time']).shift(i+4)['TOTAL_YIELD'] # 1 วันก่อนหน้า
    df2['DC_POWER_'+str(j+1)] = df2.groupby(['time']).shift(j+4)['DC_POWER'] # 1 วันก่อนหน้า
    df2['AC_POWER_'+str(j+1)] = df2.groupby(['time']).shift(j+4)['AC_POWER'] # 1 วันก่อนหน้า
    df2['AMBIENT_TEMPERATURE_'+str(j+1)] = df2.groupby(['time']).shift(j+4)['AMBIENT_TEMPERATURE'] # 1 วันก่อนหน้า
    df2['MODULE_TEMPERATURE_'+str(j+1)] = df2.groupby(['time']).shift(j+4)['MODULE_TEMPERATURE'] # 1 วันก่อนหน้า
    df2['IRRADIATION_'+str(j+1)] = df2.groupby(['time']).shift(j+4)['IRRADIATION'] # 1 วันก่อนหน้า
    
df2['mean_TOTAL_YIELD'] = (df2['TOTAL_YIELD_1'] + df2['TOTAL_YIELD_2'] + df2['TOTAL_YIELD_3']+ df2['TOTAL_YIELD_4']+ df2['TOTAL_YIELD_5']+ df2['TOTAL_YIELD_6']+ df2['TOTAL_YIELD_7'])/7
df2['mean_DC_POWER'] = (df2['DC_POWER_1'] + df2['DC_POWER_2'] + df2['DC_POWER_3']+ df2['DC_POWER_4']+ df2['DC_POWER_5']+ df2['DC_POWER_6']+ df2['DC_POWER_7'])/7
df2['mean_AC_POWER'] = (df2['AC_POWER_1'] + df2['AC_POWER_2'] + df2['AC_POWER_3']+ df2['AC_POWER_4']+ df2['AC_POWER_5']+ df2['AC_POWER_6']+ df2['AC_POWER_7'])/7
df2['mean_AMBIENT_TEMPERATURE'] = (df2['AMBIENT_TEMPERATURE_1'] + df2['AMBIENT_TEMPERATURE_2'] + df2['AMBIENT_TEMPERATURE_3']+ df2['AMBIENT_TEMPERATURE_4']+ df2['AMBIENT_TEMPERATURE_5']+ df2['AMBIENT_TEMPERATURE_6']+ df2['AMBIENT_TEMPERATURE_7'])/7
df2['mean_MODULE_TEMPERATURE'] = (df2['MODULE_TEMPERATURE_1'] + df2['MODULE_TEMPERATURE_2'] + df2['MODULE_TEMPERATURE_3']+ df2['MODULE_TEMPERATURE_4']+ df2['MODULE_TEMPERATURE_5']+ df2['MODULE_TEMPERATURE_6']+ df2['MODULE_TEMPERATURE_7'])/7
df2['mean_IRRADIATION'] = (df2['IRRADIATION_1'] + df2['IRRADIATION_2'] + df2['IRRADIATION_3']+ df2['IRRADIATION_4']+ df2['IRRADIATION_5']+ df2['IRRADIATION_6']+ df2['IRRADIATION_7'])/7


In [ ]:
data1.columns

In [ ]:
# df1[(df1['time']=='12:00')].head(10)

In [ ]:
df1.columns

In [ ]:
# df2[(df2['time']=='12:00')].head(10)

In [ ]:
df2.columns

In [ ]:
df1 = df1.dropna(axis=0)
df2 = df2.dropna(axis=0)

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df1.columns

In [ ]:
df2.columns

In [ ]:
df1_fea = df1.drop(['period', 'DATE_TIME', 'date', 'time', 'DC_POWER', 'AC_POWER',
       'DAILY_YIELD', 'AMBIENT_TEMPERATURE',
       'MODULE_TEMPERATURE', 'IRRADIATION','TOTAL_YIELD_1', 'TOTAL_YIELD_2', 'TOTAL_YIELD_3','DC_POWER_1', 'AC_POWER_1',
       'AMBIENT_TEMPERATURE_1', 'MODULE_TEMPERATURE_1', 'IRRADIATION_1',
       'DC_POWER_2', 'AC_POWER_2', 'AMBIENT_TEMPERATURE_2',
       'MODULE_TEMPERATURE_2', 'IRRADIATION_2', 'DC_POWER_3', 'AC_POWER_3',
       'AMBIENT_TEMPERATURE_3', 'MODULE_TEMPERATURE_3', 'IRRADIATION_3'],axis =1)

df2_fea = df2.drop(['period', 'DATE_TIME', 'date', 'time', 'DC_POWER', 'AC_POWER',
       'DAILY_YIELD', 'AMBIENT_TEMPERATURE',
       'MODULE_TEMPERATURE', 'IRRADIATION','TOTAL_YIELD_1', 'DC_POWER_1',
       'AC_POWER_1', 'AMBIENT_TEMPERATURE_1', 'MODULE_TEMPERATURE_1',
       'IRRADIATION_1', 'TOTAL_YIELD_2', 'DC_POWER_2', 'AC_POWER_2',
       'AMBIENT_TEMPERATURE_2', 'MODULE_TEMPERATURE_2', 'IRRADIATION_2',
       'TOTAL_YIELD_3', 'DC_POWER_3', 'AC_POWER_3', 'AMBIENT_TEMPERATURE_3',
       'MODULE_TEMPERATURE_3', 'IRRADIATION_3', 'TOTAL_YIELD_4', 'DC_POWER_4',
       'AC_POWER_4', 'AMBIENT_TEMPERATURE_4', 'MODULE_TEMPERATURE_4',
       'IRRADIATION_4', 'TOTAL_YIELD_5', 'DC_POWER_5', 'AC_POWER_5',
       'AMBIENT_TEMPERATURE_5', 'MODULE_TEMPERATURE_5', 'IRRADIATION_5',
       'TOTAL_YIELD_6', 'DC_POWER_6', 'AC_POWER_6', 'AMBIENT_TEMPERATURE_6',
       'MODULE_TEMPERATURE_6', 'IRRADIATION_6', 'TOTAL_YIELD_7', 'DC_POWER_7',
       'AC_POWER_7', 'AMBIENT_TEMPERATURE_7', 'MODULE_TEMPERATURE_7',
       'IRRADIATION_7'],axis =1)

In [ ]:
df1_fea.columns

In [ ]:
df2_fea.columns

# For n = 3

# Create n flod [for predict n = 3]

In [ ]:
#create ten flod
tenf = [df1_fea[df1_fea.index % 10 == i] for i in range(10)] 
tenf

# model regression

In [ ]:
# train = tenf[1:]
# test  = tenf[0]
# train = pd.concat(train)

# x_train = train.drop(['TOTAL_YIELD'],axis =1)
# y_train = train['TOTAL_YIELD']

# x_test = test.drop(['TOTAL_YIELD'],axis =1 )
# y_test = test['TOTAL_YIELD']
def regression_model():
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import LinearRegression
    from catboost import CatBoostRegressor
    import time
    from sklearn import metrics
    from sklearn.metrics import mean_squared_error,confusion_matrix

    RF = RandomForestRegressor()
    LR = LinearRegression()
    CB = CatBoostRegressor()

    listt = [RF,LR,CB]
    name = ['RandomForestRegressor','LinearRegression','CatBoostRegressor']
    
    return listt, name

In [ ]:
def pipeline(tenf):
    
    listt = regression_model()[0]
    name = regression_model()[1]
    
    fold = 10
    train = []
    test = []

    Train_score =[]
    Test_score=[]
    Root_Mean_Square_Error=[]

    y_test_all = []
    y_pred_final = []
    compare_scores = []


    for L in range (len(listt)):
        print(name[L])
        for i in range(fold):
            train = []
            test = []
            for j in range(fold):
                if i == j:
                    test  = tenf[i]
                elif i != j:
                    train.append(tenf[j])

            train = pd.concat(train)
            x_train = train.drop(['TOTAL_YIELD'],axis =1)
            y_train = train['TOTAL_YIELD']

            x_test = test.drop(['TOTAL_YIELD'],axis =1 )
            y_test = test['TOTAL_YIELD']


            model = listt[L]
            model.fit(x_train,y_train)

            y_pred = listt[L].predict(x_test)

            Train_score.append(listt[L].score(x_train,y_train))
            Test_score.append(listt[L].score(x_test,y_test))
            Root_Mean_Square_Error.append(mean_squared_error(y_test,y_pred)**0.5)

            y_test_all.append(y_test)
            y_pred_final.append(y_pred)

            print(str(i) + "______________________________________________")
            print('Train score : {}'.format(listt[L].score(x_train,y_train)))
            print('Test score : {}'.format(listt[L].score(x_test,y_test)))
            print('Root Mean Square Error : {}'.format(mean_squared_error(y_test,y_pred)**0.5))

        compare_scores.append([name[L],np.mean(Train_score),np.mean(Test_score),np.mean(Root_Mean_Square_Error)])
        
    return compare_scores, y_test_all, y_pred_final


# pipeline for day = 3

In [ ]:
output = pipeline(tenf)
compare_scores = output[0]
y_test_all  = output[1]
y_pred_final  = output[2]



In [ ]:
pd.DataFrame(compare_scores,columns=['Predictor','Training score','Testing score','Root_Mean_Square_Error'])

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3)
fig.suptitle('Real Vs Predict')

ax1.plot(y_test_all[0].values)
ax1.plot(y_pred_final[0])
ax1.legend(['Actual Value','Prediction'])
# ax1.title('CatBoostRegressor')

ax2.plot(y_test_all[10].values)
ax2.plot(y_pred_final[10])
ax2.legend(['Actual Value','Prediction'])
# ax2.title('CatBoostRegressor')

ax3.plot(y_test_all[20].values)
ax3.plot(y_pred_final[20])
ax3.legend(['Actual Value','Prediction'])
# ax3.title('CatBoostRegressor')

# For n = 7

# Create n flod [for predict n = 7]

In [ ]:
#create ten flod
tenf = [df2_fea[df2_fea.index % 10 == i] for i in range(10)] 
tenf

In [ ]:
output = pipeline(tenf)
compare_scores = output[0]
y_test_all  = output[1]
y_pred_final  = output[2]

In [ ]:
pd.DataFrame(compare_scores,columns=['Predictor','Training score','Testing score','Root_Mean_Square_Error'])

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3)
fig.suptitle('Real Vs Predict')

ax1.plot(y_test_all[0].values)
ax1.plot(y_pred_final[0])
ax1.legend(['Actual Value','Prediction'])
# ax1.title('CatBoostRegressor')

ax2.plot(y_test_all[10].values)
ax2.plot(y_pred_final[10])
ax2.legend(['Actual Value','Prediction'])
# ax2.title('CatBoostRegressor')

ax3.plot(y_test_all[20].values)
ax3.plot(y_pred_final[20])
ax3.legend(['Actual Value','Prediction'])
# ax3.title('CatBoostRegressor')